# Data Analysis

In [1]:
import pandas as pd
import numpy as np

# Data Preparation

In [3]:
# Import Data
data = pd.read_csv("../Thesis Data/Q1_clean.csv")

In [20]:
print('{} total loans, {:.4%} defaulted'.format(data.shape[0], np.mean(data['Default'])))

479862 total loans, 0.0129% defaulted


In [9]:
# Reset index to Loan ID
data.drop('Unnamed: 0', axis=1, inplace=True)
data.set_index('LoanID', inplace=True)

In [23]:
print(data.columns.values)

['Channel' 'SellerName' 'OrInterestRate' 'OrUPB' 'OrLoanTerm' 'OrDate'
 'FirstPayDate' 'OrLTV' 'OrCLTV' 'NumBorrow' 'OrDTIRat' 'OrCreditScore'
 'FTHomeBuyer' 'LoanPurpose' 'PropertyType' 'NumUnits' 'OccStatus'
 'PropertyState' 'Zip' 'MortInsPerc' 'ProductType' 'CoCreditScore'
 'MortInsType' 'RelMortInd' 'Default' 'CoCreditor']


Convert the following:
- FTHomeBuyer (Y and N)
- LoanPurpose
- Split date month and year



# SMOTE

In [35]:
data.OccStatus.head()

LoanID
100002130634    1
100003080256    2
100003722619    1
100004837443    1
100007361597    1
Name: NumUnits, dtype: int64